# Lista de Exercícios 4
**Aluno**: Vítor Gabriel Reis Caitité

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from scipy import stats
import statsmodels.api as sm

In [2]:
dataset = pd.read_csv('~/Documents/UFMG/Doutorado/materias/Modelos_de_Regressao/atividade_4/boston_house_prices.csv', header=0, sep=",", engine='python')
dataset.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,538.0,6575.0,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,469.0,6421.0,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,469.0,7185.0,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,458.0,6998.0,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,458.0,7147.0,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
# Selecionando as variáveis de ambiente requeridas:
X = dataset.drop(["MEDV"], axis=1)
y = dataset["MEDV"]

## Modelo de Regressão Linear Múltipla utilizando o pacote "statsmodels"

Ajuste do modelo de regressão linear múltipla, considerando as variáveis independentes:

![table1](independentes.png)

e a variável dependente:

![table2](y.png)

In [4]:
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit() 
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     71.40
Date:                Fri, 08 Nov 2024   Prob (F-statistic):          2.83e-104
Time:                        20:12:38   Log-Likelihood:                -1572.0
No. Observations:                 506   AIC:                             3172.
Df Residuals:                     492   BIC:                             3231.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         49.4240      3.166     15.610      0.0

### Remoção de Variáveis do Modelo

Remoção das variáveis  do modelo com P-valor > 0.05.

In [5]:
X.drop(["ZN", "INDUS", "NOX", "CHAS", "RM", "DIS"], axis=1, inplace=True)
X.head()

,const,CRIM,AGE,RAD,TAX,PTRATIO,B,LSTAT
0,1.0,0.00632,65.2,1,296,15.3,396.90,4.98
1,1.0,0.02731,78.9,2,242,17.8,396.90,9.14
2,1.0,0.02729,61.1,2,242,17.8,392.83,4.03
3,1.0,0.03237,45.8,3,222,18.7,394.63,2.94
4,1.0,0.06905,54.2,3,222,18.7,396.90,5.33


#### Geração de um novo modelo para o novo dataset selecionado

In [6]:
model = sm.OLS(y, X)
results = model.fit() 
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.642
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     127.5
Date:                Fri, 08 Nov 2024   Prob (F-statistic):          9.34e-107
Time:                        20:12:38   Log-Likelihood:                -1580.5
No. Observations:                 506   AIC:                             3177.
Df Residuals:                     498   BIC:                             3211.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         53.5403      2.666     20.083      0.0

## Replicando os resultados utilizando abordagem matricial

A regressão linear múltipla pode ser representada pela equação:

$y=X\beta+\epsilon$, 

sendo:

- $y$ é o vetor da variável dependente;
- $X$ é a matriz de variáveis independentes (com a constante);
- $\beta$ é o vetor de coeficientes (estimadores);
- $\epsilon$ é o vetor de resíduos.



### Estimadores ($\hat\beta$)

$\hat{\beta} = (X^T X)^{-1} X^T y$

In [7]:
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
print(round(beta_hat, 4))

0    53.5403
1     0.0004
2     0.0424
3     0.3511
4    -0.0163
5    -1.2334
6     0.0090
7    -0.8789
dtype: float64


É possível perceber que esses valores são os mesmos encontrados pelo modelo desenvolvido utilizando o pacote "statsmodels".

### Resíduos ($\epsilon$)

$\epsilon = y - \hat{y}$

sendo:

- $\hat{y} = X \hat{\beta}$

In [8]:
y_hat = X.to_numpy() @ beta_hat
epsilon = y - y_hat
print(round(epsilon, 4))

0      -8.1745
1      -5.6459
2       3.7541
3       2.5616
4       7.0856
        ...   
501     0.8827
502    -1.8024
503    -2.1320
504    -3.1905
505   -11.7309
Name: MEDV, Length: 506, dtype: float64


Calculando o somatório da diferença entre os resíduos encontrados pelo modelo do pacote e o implementado pode-se observar que o resultado é 0, como esperado.

In [9]:
print(round(sum(epsilon - results.resid), 6))

0.0


### R-quadrado (R^2)


$R^2 = 1 - \frac{\text{SSE}}{\text{SST}}$

onde:

- Soma dos quadrados dos resíduos (SSE):

  $\text{SSE} = \sum (y - \hat{y})^2$

- Soma total dos quadrados (SST):
  
  $\text{SST} = \sum (y - \bar{y})^2$

In [10]:
SSE = np.sum(epsilon**2)
SST = np.sum((y - np.mean(y))**2)
R2 = 1 - SSE / SST
print("R^2:", round(R2,3))

R^2: 0.642


Esse é exatamente o mesmo valor encontrado pelo modelo desenvolvido utilizando o pacote "statsmodels".

### Desvio-padrão dos estimadores

Tem-se que $\text{Cov}(\hat{\beta}) = \sigma^2 (X^T X)^{-1}$

onde:
$\sigma^2 = \frac{\text{SSE}}{n - p}$

Assim, o desvio padrão de cada estimador será a raiz quadrada dos elementos diagonais dessa matriz. Como pode-se ver abaixo, os valores encontrado correspondem aos encontrados utilizando o pacote "statsmodels".

In [11]:
n, p = X.shape
sigma2 = SSE / (n - p)
cov_beta_hat = sigma2 * np.linalg.inv(X.T @ X)
std_beta_hat = np.sqrt(np.diag(cov_beta_hat)) 
np.set_printoptions(suppress=True, precision=3)
print("Desvio-padrão dos estimadores:", std_beta_hat)

Desvio-padrão dos estimadores: [2.666 0.    0.011 0.07  0.004 0.132 0.003 0.048]


### Desvio-padrão dos resíduos

$\text{Std residual} = \sqrt{\frac{\text{SSE}}{n - p}}$

sendo que:

- "n" é o número de observações,
- "p" é o número de parâmetros (incluindo o intercepto).

In [12]:
std_epsilon = np.sqrt(SSE / (n - p))
print("Desvio-padrão residual:", std_epsilon)

Desvio-padrão residual: 5.542884571805483


Pode-se perceber, abaixo, que esse é o mesmo desvio padrão residual encontrado considerando o modelo gerado a partir do pacote "statsmodels".

In [13]:
# Residual Standard Error of the model
np.sqrt(results.scale)

5.542884571805483

### P-Valor

Os p-valores são calculados com base no teste t-Student para cada coeficiente, sendo:

$t_i = \frac{\hat{\beta}_i}{\text{desvio-padrão de } \hat{\beta}_i}$.

Assim:

$p\text{-valor} = 2 \times \left(1 - F_t(|t|)\right)$

onde:

- $F_t$ é a função de distribuição acumulada (CDF) da distribuição t com n−p graus de liberdade;
- $∣t∣$ é o valor absoluto do valor t calculado para o coeficiente.


In [14]:
t_values = beta_hat/std_beta_hat
p_values = [2 * (1 - stats.t.cdf(np.abs(t), df=n - p)) for t in t_values]
np.set_printoptions(suppress=True, precision=3)
print("P-valores:", (np.array(p_values)))

P-valores: [0.    0.049 0.    0.    0.    0.    0.004 0.   ]


Os valores acima correspondem exatamente aos p-valores encontrados pelo pacote "statsmodels".

### Estatística $F$

Por fim, decidiu-se também verificar a estatística F, como explicado na seção 3.7.1 da disciplina.

$F = \frac{\left(\frac{\text{SSR}}{p - 1}\right)}{\left(\frac{\text{SSE}}{n - p}\right)}$

sendo:

- $\text{SSR} = \sum (\hat{y} - \bar{y})^2$;
- $\text{SSE} = \sum (y - \hat{y})^2$;
- p: número de parâmetros do modelo (com o intercepto);
- n: número de observações.

In [15]:
SSR = np.sum((y_hat - np.mean(y))**2)
f_statistic = (SSR / (p - 1)) / (SSE / (n - p))
print("F-statistic:", round(f_statistic, 1))

F-statistic: 127.5


Novamente, percebe-se que o mesmo valor para a estatística F foi encontrado utilizando o pacote "statsmodels".